In [32]:
# imports
import pandas as pd
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,accuracy_score,log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
import warnings
from sklearn.svm import SVC
warnings.filterwarnings('ignore')



In [33]:
glass_df = pd.read_csv(r'..\Cases\Glass Identification\Glass.csv')
glass_df.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,building_windows_float_processed
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,building_windows_float_processed
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,building_windows_float_processed
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,building_windows_float_processed
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,building_windows_float_processed


In [34]:
X= glass_df.drop(columns=['Type'])
y= glass_df['Type']
scaler=StandardScaler()
le= LabelEncoder()

y= le.fit_transform(y)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=25,stratify=y)
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [35]:
log_reg_l1=LogisticRegression(penalty='l2')
svc_radial = SVC(kernel='rbf',probability=True,random_state=25)
svc_lin = SVC(kernel='linear',probability=True,random_state=25)
dtree=DecisionTreeClassifier()


scaler=StandardScaler().set_output(transform='pandas')
p1=Pipeline([('scaler',scaler),('SVC_radial',svc_radial)])
p2=Pipeline([('scaler',scaler),('SVC_linear',svc_lin)])

In [45]:
from sklearn.ensemble import BaggingClassifier
from tqdm import tqdm
models=[log_reg_l1,p1,p2,dtree]
n_estm=[10,25,50,75,100,125,150]
scores=[]
for n in tqdm(range(len(n_estm))):
    for m in models:
        bagg=BaggingClassifier(estimator=m,n_estimators=n_estm[n],random_state=25)
        bagg.fit(X_train,y_train)
        y_pred_proba=bagg.predict_proba(X_test)
        y_pred=bagg.predict(X_test)
        scores.append([m,n_estm[n],log_loss(y_test,y_pred_proba),accuracy_score(y_test,y_pred)])
scores_df=pd.DataFrame(scores,columns=['models','n_estimators','log_loss','accuracy'])
scores_df.sort_values(by='log_loss',ascending=True,inplace=True)
scores_df.head()

100%|██████████| 7/7 [00:48<00:00,  6.98s/it]


,models,n_estimators,log_loss,accuracy
23,DecisionTreeClassifier(),125,0.637209,0.815385
19,DecisionTreeClassifier(),100,0.638158,0.815385
27,DecisionTreeClassifier(),150,0.640763,0.815385
25,"(StandardScaler(), SVC(probability=True, rando...",150,0.806131,0.723077
21,"(StandardScaler(), SVC(probability=True, rando...",125,0.809457,0.723077
